In [ ]:
!pip install dateparser> /dev/null
!pip install transformers > /dev/null
!pip install hdbscan > /dev/null
!pip install umap-learn > /dev/null

# Глубинное обучение в IT мониторинге

На основе лекции и семинара по Bert и Anomaly Detection я написал приложение которое анализирует лог файл и находит аномальные события за указанный период времени. Использовал библиотеки Transformers и HDBSCAN.

Импортирую и устанавливаю библиотеки, определяю класс и функции.

In [ ]:
from tqdm import tqdm
import logging
import re
import dateparser
import string
import pandas as pd
import configparser
import datetime

import hdbscan

import numpy as np
import torch
import transformers as ppb

config = configparser.ConfigParser()
config.read('/joyce/app/joyce.conf')

logger = logging.getLogger('joyce-app')

DEBUG = True
LEN_TIMESTAMP = 15

if DEBUG == 'True':
    logger.setLevel(logging.DEBUG)
else:
    logger.setLevel(logging.INFO)

model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+=', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\d', '', text)
    return text

class Event2Vec:
    """
    len_timestamp:
        /var/log/messages - 15

    date_time_str:
        '05/12/21 03:00' Example: if problem in 12 Dec 03:24'
    """
    def __init__(self, len_timestamp: int, date_time_str='05/12/21 03:00', \
                 tokenizer=tokenizer, model=model):
        
        self.len_timestamp = len_timestamp
        self.tokenizer = tokenizer
        self.model = model
        self.date_end = dateparser.parse(date_time_str)
        self.date_start = self.date_end - datetime.timedelta(days=2)
        self.features = None
        self.max_len = 40

    
    def look(self, logfile: str = 'andarta5_messages.txt'):
        # egrep -vi "Created slice|Removed slice|Started Session"
        self.raw_data = open(logfile).readlines()
        self.len_hostname = len(self.raw_data[0][self.len_timestamp + 1:].split()[0])
        self.df = pd.DataFrame(self.raw_data, columns=['text'])

        self.df['clean']  = self.df.text.str[self.len_timestamp + self.len_hostname:].apply(lambda x:clean_text(x))
        self.date_event_raw = [x[:self.len_timestamp] for x in self.raw_data]
        self.df['date'] = [dateparser.parse(x) for x in self.date_event_raw]
        self.mask = (self.df['date'] > self.date_start) & (self.df['date'] <= self.date_end)
        self.mask_x = ( (self.df['date'] > self.date_end) & (self.df['date'] <= (self.date_end + datetime.timedelta(hours=1))))
        self.df_previous = self.df.loc[self.mask]
        self.df_x = self.df.loc[self.mask_x]

        self.epochs = [self.df_previous[x:x+500] for x in range(0, len(self.df_previous), 500)]

        for epoch in tqdm(self.epochs):

            self.tokenized = epoch['clean'].apply(lambda x: self.tokenizer.encode(x, add_special_tokens=True))
            input_ids, attention_mask = self._get_attention_mask(self.tokenized)

            with torch.no_grad():
                last_hidden_states = self.model(input_ids, attention_mask=attention_mask)
            
            if self.features is None:
                self.features = last_hidden_states[0][:,0,:].numpy()
            else:
                self.features = np.append(self.features, last_hidden_states[0][:,0,:].numpy(), 0)

        

        self.cluster_model = hdbscan.HDBSCAN(min_cluster_size=2,
                          metric='euclidean',                      
                          cluster_selection_method='eom', prediction_data=True).fit(self.features)

        self.df_previous['labels'] = self.cluster_model.labels_


        self.tokenized_x = self.df_x['clean'].apply(lambda x: self.tokenizer.encode(x, add_special_tokens=True))
        
        input_ids, attention_mask = self._get_attention_mask(self.tokenized_x)

        with torch.no_grad():
            last_hidden_states = self.model(input_ids, attention_mask=attention_mask)
        
        self.df_x['labels'] = hdbscan.approximate_predict(self.cluster_model, last_hidden_states[0][:,0,:].numpy())[0]
        self.df_x[self.df_x.labels == -1][['date', 'text']].to_csv('anomaly')
        return self.df_x[self.df_x.labels == -1][['date', 'text']]

    def _get_attention_mask(self, tokenized):

        padded = np.array([i + [0]*(self.max_len-len(i)) for i in tokenized.values])

        attention_mask = np.where(padded != 0, 1, 0)
        input_ids = torch.tensor(padded[:])  
        attention_mask = torch.tensor(attention_mask[:])
        
        return input_ids, attention_mask

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Создаю объект класса, передаю ему дату и время в которое предположительно произошло аномальное событие. После анализа, модель возвращает дата фрэйм, с событиями в данное время, которые HDBSCAN посчитал аномальными, по сравнению с записями ранее. BERT я использовал для получения эмбедингов предложений:

In [ ]:
event2vec = Event2Vec(len_timestamp=LEN_TIMESTAMP, date_time_str='2022-12-7 14:00:00')
event2vec.look(logfile="messages")

grep: pvli-crm0054_messages.txt: No such file or directory


100%|██████████| 14/14 [05:40<00:00, 24.33s/it]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,date,text
8388,2022-12-07 14:05:28,Dec 7 14:05:28 andarta5 chronyd[924]: Selecte...
8446,2022-12-07 14:28:10,Dec 7 14:28:10 andarta5 python: ansible-slurp...
